# Movie sentiment analysis

**Data:**

The data source is Movie Sentiment project from Kaggle, including train and test datasets; In trainning data, there is a Phrase column and a Sentiment column as the result score from 0 to 4 (negative to positve); In test data, there is a Phrase column for us to analyze the sentiment of each phrase.

**Goal:**


Perform different deep learning models (ANN, CNN, RNN-LSTM) in NLP analysis on trainning data and compare model performances, and predict Phrase sentiment on test data. 

**Highlight:** 

- **Show how to prepare data for deep learning in NLP analysis**
- **Compare different deep learning models in NLP analysis**

**Reference**

Kaggel notebook: https://www.kaggle.com/chiranjeevbit/movie-review-prediction


Tensorflow helpfile: https://www.tensorflow.org/tutorials/structured_data/imbalanced_data

## Explore Data Analysis

In [ ]:
## Import basic packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
## Read data
train = pd.read_csv('/kaggle/input/sentiment-analysis-on-movie-reviews/train.tsv.zip',sep="\t") 
test = pd.read_csv('/kaggle/input/sentiment-analysis-on-movie-reviews/test.tsv.zip',sep="\t") 

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
train.Sentiment.value_counts()

In [ ]:
train.info()

In [ ]:
## Show the number of class distributed
plt.figure(figsize=(10,5))
ax=plt.axes()
ax.set_title('Number of sentiment class')
sns.countplot(x=train.Sentiment,data=train)

## Remove punctuation

In [ ]:
import string
string.punctuation

In [ ]:
train['Phrase1']=train.Phrase.apply(lambda x: x.translate(str.maketrans('','',string.punctuation)).lower())
test['Phrase1']=test.Phrase.apply(lambda x: x.translate(str.maketrans('','',string.punctuation)).lower())

In [ ]:
train.head()

## Split train data into train and validation datasets

In [ ]:
train_shuffle=train.sample(frac=1, random_state=1)

In [ ]:
split=int(0.7*train.shape[0])
train_data=train_shuffle[0:split]
valid_data=train_shuffle[split:]

## Use Glove for Wordembedding

In [ ]:
words=set()
word_to_vec_map={}
with open('/kaggle/input/glove-50d/glove.6B.50d.txt','r',encoding='UTF-8') as f:
    for line in f:
        value=line.strip().split()
        word=value[0]
        words.add(word)
        word_to_vec_map[word]=np.array(value[1:],dtype=np.float64)

## Data preparation for Modelling

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import regularizers
from tensorflow import keras

In [ ]:
## fit train data into vector
tokenizer=Tokenizer()
tokenizer.fit_on_texts(train.Phrase1)
train_sequences=tokenizer.texts_to_sequences(train_data.Phrase1)
valid_sequences=tokenizer.texts_to_sequences(valid_data.Phrase1)
test_sequences=tokenizer.texts_to_sequences(test.Phrase1)

In [ ]:
## Pad sequence data with 0 for same length
maxlen=train.Phrase1.apply(lambda x: len(x)).max()
train_padded=pad_sequences(train_sequences,maxlen=maxlen,padding='post',truncating='post')
valid_padded=pad_sequences(valid_sequences,maxlen=maxlen,padding='post',truncating='post')
test_padded=pad_sequences(test_sequences,maxlen=maxlen,padding='post',truncating='post')
print(train_padded.shape,valid_padded.shape,test_padded.shape)

In [ ]:
word_index=tokenizer.word_index
vocab_size=len(word_index)
embedding_matrix=np.zeros((vocab_size+1,50))   ## for unknown word, add 1 in vocab_size

In [ ]:
## Use Glove wordembedding
for word, i in word_index.items():
    embedding_vector=word_to_vec_map.get(word)
    if embedding_vector is not None:
        embedding_matrix[i]=embedding_vector

In [ ]:
train_x=np.array(train_padded)
train_y=np.array(train_data.Sentiment)
valid_x=np.array(valid_padded)
valid_y=np.array(valid_data.Sentiment)
test_x=np.array(test_padded)

In [ ]:
N_TRAIN=len(train_x)
BATCH_SIZE=256
STEPS_PER_EPOCH=N_TRAIN/BATCH_SIZE

## decrease the learning rate when epoch increase.
lr_decay=tf.keras.optimizers.schedules.InverseTimeDecay(
    1e-2,
    decay_steps=STEPS_PER_EPOCH*100,
    decay_rate=1,
    staircase=False
)
histories={}

## Deep learning model comparision

### ANN

In [ ]:
## Simple NN model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size+1,50,input_length=maxlen, weights=[embedding_matrix],trainable=False),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')
])


In [ ]:
model.compile(loss='sparse_categorical_crossentropy',optimizer=keras.optimizers.Adam(lr_decay),metrics='accuracy')
model.summary()

In [ ]:
histories['ANN']=model.fit(train_x, train_y, epochs = 10,validation_data=(valid_x,valid_y), batch_size = 128)

In [ ]:
colors=['blue','green']
def plot_metrics(history):
  metrics =  ['loss', 'accuracy']
  for n, metric in enumerate(metrics):
    name = metric.replace("_"," ").capitalize()
    plt.subplot(1,2,n+1)
    plt.plot(history.epoch,  history.history[metric], color=colors[0], label='Train')
    plt.plot(history.epoch, history.history['val_'+metric],
             color=colors[1], linestyle="--", label='Val')
    plt.xlabel('Epoch')
    plt.ylabel(name)
    plt.legend()

In [ ]:
plt.figure(figsize=(20,7))
plot_metrics(histories['ANN'])

### Convolution NN

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size+1,50,input_length=maxlen, weights=[embedding_matrix],trainable=False),
    #tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Conv1D(128, 2, padding='same',activation='relu'),
    #tf.keras.layers.Dropout(0.5),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(5, activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy',optimizer=keras.optimizers.Adam(lr_decay),metrics='accuracy')
model.summary()

In [ ]:
histories['CNN']=model.fit(train_x, train_y, epochs = 10,validation_data=(valid_x,valid_y), batch_size = 256)

In [ ]:
plt.figure(figsize=(20,7))
plot_metrics(histories['CNN'])

### RNN-LSTM, GRU

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size+1,50,input_length=maxlen, weights=[embedding_matrix],trainable=False),
    #tf.keras.layers.Dropout(0.5),
    tf.keras.layers.LSTM(256, return_sequences=True),
    tf.keras.layers.LSTM(256),
    #tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(125, activation='relu'),
    #tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(5, activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy',optimizer=keras.optimizers.Adam(lr_decay),metrics='accuracy')
model.summary()


In [ ]:
histories['LSTM_1']=model.fit(train_x, train_y, epochs = 10,validation_data=(valid_x,valid_y), batch_size = 128)

In [ ]:
plt.figure(figsize=(20,7))
plot_metrics(histories['LSTM_1'])

### Bidirectional LSTM

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size+1,50,input_length=maxlen, weights=[embedding_matrix],trainable=False),
    #tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    #tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(5, activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy',optimizer=keras.optimizers.Adam(lr_decay),metrics='accuracy')
model.summary()

In [ ]:
histories['LSTM_2']=model.fit(train_x, train_y, epochs = 10,validation_data=(valid_x,valid_y), batch_size = 256)

In [ ]:
plt.figure(figsize=(20,7))
plot_metrics(histories['LSTM_2'])

Bidirectional LSTM tunning

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size+1,50,input_length=maxlen, weights=[embedding_matrix],trainable=False),
    #tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    #tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(5, activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy',optimizer=keras.optimizers.Adam(lr_decay),metrics='accuracy')
model.summary()

In [ ]:
histories['LSTM_3']=model.fit(train_x, train_y, epochs = 10,validation_data=(valid_x,valid_y), batch_size = 256)

In [ ]:
plt.figure(figsize=(20,7))
plot_metrics(histories['LSTM_3'])

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size+1,50,input_length=maxlen, weights=[embedding_matrix],trainable=False),
    #tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    #tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(5, activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy',optimizer=keras.optimizers.Adam(lr_decay),metrics='accuracy')
model.summary()

In [ ]:
histories['LSTM_4']=model.fit(train_x, train_y, epochs = 10,validation_data=(valid_x,valid_y), batch_size = 256)

In [ ]:
plt.figure(figsize=(20,7))
plot_metrics(histories['LSTM_4'])

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size+1,50,input_length=maxlen, weights=[embedding_matrix],trainable=False),
    #tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(5, activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy',optimizer=keras.optimizers.Adam(lr_decay),metrics='accuracy')
model.summary()

In [ ]:
histories['LSTM_5']=model.fit(train_x, train_y, epochs = 10,validation_data=(valid_x,valid_y), batch_size = 256)

In [ ]:
plt.figure(figsize=(20,7))
plot_metrics(histories['LSTM_5'])

## Submission

In [ ]:
test_y=model.predict_classes(test_x)

In [ ]:
sub_file = pd.read_csv('/kaggle/input/sentiment-analysis-on-movie-reviews/sampleSubmission.csv',sep=',')
sub_file.Sentiment=test_y
sub_file.to_csv('Submission.csv',index=False)